In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np

import warnings
import datetime as dt
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [2]:
events = pd.read_csv('data/events_up_to_01062018.csv', low_memory = False)
events.person.describe()

count      2341681
unique       38829
top       c76b8417
freq          4438
Name: person, dtype: object

In [3]:
prediction_set = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory = False)
training_set = pd.read_csv('data/labels_training_set.csv', low_memory=False)

print('PREDICCION ' , prediction_set.person.describe())
print('ENTRENAMIENTO ', training_set.person.describe())
print('ENTRENAMIENTO 1s ', training_set[training_set['label'] == 1].shape)
print('ENTRENAMIENTO 0s ', training_set[training_set['label'] == 0].shape)

prediction_set = prediction_set.merge(events, on = 'person', how = 'inner')
prediction_set = prediction_set[(prediction_set['event'] == 'visited site') | (prediction_set['event'] == 'viewed product') | (prediction_set['event'] == 'conversion') | (prediction_set['event'] == 'checkout') | (prediction_set['event'] == 'brand listing') | (prediction_set['event'] == 'generic listing') | (prediction_set['event'] == 'lead') | (prediction_set['event'] == 'staticpage') ]                                                                       

training_set = events.merge(training_set, on = 'person', how = 'inner')

info = training_set[['person']]
info['frecuencia'] = 1
data = pd.DataFrame({'frecuencia': info.groupby('person').aggregate(sum)['frecuencia']}).reset_index()

training_set = training_set.merge(data, on = 'person', how = 'left')
training_set= training_set[(training_set['label'] == 1) | ((training_set['frecuencia'] <= 100) & (training_set['label'] == 0))]


PREDICCION  count        19415
unique       19415
top       765e0fb6
freq             1
Name: person, dtype: object
ENTRENAMIENTO  count        19414
unique       19414
top       5f434220
freq             1
Name: person, dtype: object
ENTRENAMIENTO 1s  (980, 2)
ENTRENAMIENTO 0s  (18434, 2)


In [4]:
# del info
# del data
del events

In [5]:
prediction_set.person.describe()

count       953099
unique       19415
top       5059f7fd
freq          2932
Name: person, dtype: object

In [6]:
training_set.person.describe()

count       525152
unique       16779
top       c7cc2c23
freq          1561
Name: person, dtype: object

In [7]:
#ELEGIR SET

In [8]:
#PREDICCION
sets = prediction_set
cols_drop=['url','skus', 'city','search_engine','campaign_source','channel']
cols = ['event','fecha','person', 'sku', 'model', 'condition', 'storage', 'color']
cols_visit = ['event','fecha','person','new_vs_returning','region','country','device_type','screen_resolution','operating_system_version','browser_version']
cols_x = ['fecha_x','person','sku_x', 'model_x', 'condition_x', 'storage_x', 'color_x','compras_prod_x','vistas_prod_x', 'otros_x']
rename = {'fecha_x': 'fecha', 'model_x': 'model','sku_x':'sku', 'condition_x': 'condition', 'storage_x': 'storage', 'color_x': 'color', 'compras_prod_x': 'compras_prod', 'vistas_prod_x': 'vistas_prod' ,'otros_x': 'otros'}

In [8]:
#ENTRENAMIENTO
sets = training_set
cols_drop=['url','skus', 'city','search_engine','campaign_source','channel']
cols = ['event','fecha','person', 'sku', 'model', 'condition', 'storage', 'color', 'label']
cols_visit = ['event','fecha','person','new_vs_returning','region','country','device_type','screen_resolution','operating_system_version','browser_version', 'label']
cols_x = ['fecha_x','person','sku_x', 'model_x', 'condition_x', 'storage_x', 'color_x','compras_prod_x','vistas_prod_x', 'otros_x', 'label_x']
rename = {'fecha_x': 'fecha', 'model_x': 'model','sku_x':'sku', 'condition_x': 'condition', 'storage_x': 'storage', 'color_x': 'color', 'compras_prod_x': 'compras_prod', 'vistas_prod_x': 'vistas_prod' ,'otros_x': 'otros', 'label_x': 'label'}

In [9]:
del prediction_set
del training_set

# Set Datos

# #Analisis TP1

In [9]:
sets.columns

Index(['person', 'timestamp', 'event', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version'],
      dtype='object')

In [10]:
sets.drop(cols_drop, inplace=True, axis=1)

In [11]:
#Cambiamos el tipo de dato de fecha de timestamp a datetime para un mejor analisis y renombramos a "fecha"
sets[['timestamp']] = sets[['timestamp']].apply(pd.to_datetime)
sets.rename({
    'timestamp' : 'fecha',
}, axis=1, inplace=True)
sets.head()

,person,fecha,event,sku,model,condition,storage,color,search_term,staticpage,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,4886f805,2018-05-18 00:11:59,viewed product,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4886f805,2018-05-18 00:30:30,viewed product,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4886f805,2018-05-18 00:11:56,checkout,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4886f805,2018-05-18 00:11:35,viewed product,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4886f805,2018-05-18 00:11:53,viewed product,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


vamos a crear las columnas compras_prod, vistas_prod, otros para productos y visitas_sitio para persona, de la forma de One Hot Encoder

In [12]:
#Detalles de Productos que fueron comprados
productos = sets.loc[:, cols]
productos_comprados = productos[(productos['event'] == 'conversion') | (productos['event'] == 'checkout')]
productos_comprados.drop(['event'], inplace=True, axis=1)
productos_comprados['compras_prod'] = 1
productos_comprados['vistas_prod'] = 0
productos_comprados['otros'] = 0
productos_comprados.head()

,fecha,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros
3,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,1,0,0
56,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,1,0,0
89,2018-05-10 00:39:51,0297fc1e,7029.0,iPhone 6S Plus,Bom,16GB,Cinza espacial,1,0,0
313,2018-02-25 18:38:00,0297fc1e,1342.0,iPhone 6,Muito Bom,64GB,Dourado,1,0,0
339,2018-01-31 16:12:48,0297fc1e,2834.0,iPhone 6,Bom,64GB,Dourado,1,0,0


In [13]:
productos_comprados.count()

fecha           36303
person          36303
sku             36303
model           36303
condition       36303
storage         36303
color           36303
compras_prod    36303
vistas_prod     36303
otros           36303
dtype: int64

In [14]:
print('1s ', productos_comprados[productos_comprados['label'] == 1].shape)
print('0s ', productos_comprados[productos_comprados['label'] == 0].shape)

KeyError: 'label'

In [15]:
# productos_comprados = productos_comprados.drop_duplicates()
# productos_comprados.count()

In [16]:
#Detalles de Productos que fueron visitados
productos_visitados = productos.loc[productos.event == 'viewed product']
productos_visitados.drop(['event'], inplace=True, axis=1)
productos_visitados['compras_prod'] = 0
productos_visitados['vistas_prod'] = 1
productos_visitados['otros'] = 0
productos_visitados.head()

,fecha,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros
0,2018-05-18 00:11:59,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,0,1,0
1,2018-05-18 00:30:30,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,0,1,0
4,2018-05-18 00:11:35,4886f805,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,0,1,0
6,2018-05-18 00:11:53,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,0,1,0
9,2018-05-18 00:11:16,0297fc1e,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,0,1,0


In [17]:
productos_visitados.count()

fecha           618891
person          618891
sku             618891
model           618891
condition       618891
storage         618891
color           618891
compras_prod    618891
vistas_prod     618891
otros           618891
dtype: int64

In [19]:
print('1s ', productos_visitados[productos_visitados['label'] == 1].shape)
print('0s ', productos_visitados[productos_visitados['label'] == 0].shape)

1s  (53773, 11)
0s  (194154, 11)


In [20]:
# productos_visitados = productos_comprados.drop_duplicates()
# productos_visitados.count()

In [18]:
# Eliminamos de productos visitados las personas que estan en productos comprados
productos_visitados = productos_visitados.merge(productos_comprados, on='person', how='left')
productos_visitados = productos_visitados[productos_visitados['fecha_y'].isnull()]
productos_visitados = productos_visitados.loc[:,cols_x]
productos_visitados.rename(columns=rename, inplace=True)
productos_visitados.head()

,fecha,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros
6574,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,0,1,0
6575,2018-05-18 00:25:27,686c49c9,9946.0,iPhone 7,Excelente,128GB,Preto Matte,0,1,0
6576,2018-05-18 01:05:31,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,0,1,0
6577,2018-05-15 20:11:48,686c49c9,10030.0,iPhone 7,Excelente,128GB,Prateado,0,1,0
6578,2018-05-15 20:10:28,686c49c9,6845.0,iPhone 6S,Excelente,16GB,Dourado,0,1,0


In [19]:
productos_visitados.count()

fecha           58106
person          58106
sku             58106
model           58106
condition       58106
storage         58106
color           58106
compras_prod    58106
vistas_prod     58106
otros           58106
dtype: int64

In [24]:
print('1s ', productos_visitados[productos_visitados['label'] == 1].shape)
print('0s ', productos_visitados[productos_visitados['label'] == 0].shape)

1s  (13707, 11)
0s  (22765, 11)


In [20]:
otros = sets.loc[:, cols]
otros = otros[(otros['event'] == 'brand listing') | (otros['event'] == 'generic listing') | (otros['event'] == 'lead') | (otros['event'] == 'staticpage')]
otros.drop(['event'], inplace=True, axis=1)
otros['compras_prod'] = 0
otros['vistas_prod'] = 0
otros['otros'] = 1
otros.head()

,fecha,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros
7,2018-05-18 00:07:22,4886f805,NaN,NaN,NaN,NaN,NaN,0,0,1
455,2018-05-17 19:59:37,0297fc1e,NaN,NaN,NaN,NaN,NaN,0,0,1
456,2018-05-21 23:08:54,0297fc1e,NaN,NaN,NaN,NaN,NaN,0,0,1
457,2018-03-20 16:39:59,0297fc1e,NaN,NaN,NaN,NaN,NaN,0,0,1
458,2018-04-29 18:41:44,0297fc1e,NaN,NaN,NaN,NaN,NaN,0,0,1


In [21]:
otros.count()

fecha           196135
person          196135
sku                  0
model              552
condition            0
storage              0
color                0
compras_prod    196135
vistas_prod     196135
otros           196135
dtype: int64

In [27]:
print('1s ', otros[otros['label'] == 1].shape)
print('0s ', otros[otros['label'] == 0].shape)

1s  (19452, 11)
0s  (68206, 11)


In [28]:
# otros = otros.drop_duplicates()
# otros.count()

In [22]:
# Eliminamos de otros las personas que estan en productos comprados
otros = otros.merge(productos_comprados, on='person', how='left')
otros = otros[otros['fecha_y'].isnull()]
otros = otros.loc[:,cols_x]
otros.rename(columns=rename, inplace=True)
otros.head()

,fecha,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros
523,2018-05-07 12:45:15,686c49c9,NaN,NaN,NaN,NaN,NaN,0,0,1
524,2018-05-07 12:39:53,686c49c9,NaN,NaN,NaN,NaN,NaN,0,0,1
525,2018-05-07 12:41:38,686c49c9,NaN,NaN,NaN,NaN,NaN,0,0,1
526,2018-05-07 12:43:42,686c49c9,NaN,NaN,NaN,NaN,NaN,0,0,1
527,2018-05-07 12:43:43,686c49c9,NaN,NaN,NaN,NaN,NaN,0,0,1


In [24]:
otros.count()

fecha           18472
person          18472
sku                 0
model              58
condition           0
storage             0
color               0
compras_prod    18472
vistas_prod     18472
otros           18472
dtype: int64

In [31]:
print('1s ', otros[otros['label'] == 1].shape)
print('0s ', otros[otros['label'] == 0].shape)

1s  (4867, 11)
0s  (8904, 11)


In [25]:
# Eliminamos de otros las personas que estan en productos visitados
otros = otros.merge(productos_visitados, on='person', how='left')
otros = otros[otros['fecha_y'].isnull()]
otros = otros.loc[:,cols_x]
otros.rename(columns=rename, inplace=True)
otros.head()

,fecha,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros
60413,2018-05-28 12:36:14,df80a783,NaN,NaN,NaN,NaN,NaN,0,0,1
60414,2018-05-28 22:28:24,df80a783,NaN,NaN,NaN,NaN,NaN,0,0,1
60415,2018-05-29 14:47:28,df80a783,NaN,NaN,NaN,NaN,NaN,0,0,1
63146,2018-05-19 16:30:03,5437de8d,NaN,NaN,NaN,NaN,NaN,0,0,1
69477,2018-02-07 13:52:08,626d2cd6,NaN,NaN,NaN,NaN,NaN,0,0,1


In [26]:
otros.count()

fecha           590
person          590
sku               0
model             5
condition         0
storage           0
color             0
compras_prod    590
vistas_prod     590
otros           590
dtype: int64

In [34]:
print('1s ', otros[otros['label'] == 1].shape)
print('0s ', otros[otros['label'] == 0].shape)

1s  (46, 11)
0s  (522, 11)


In [27]:
#Caracterisitcas del dispositivos con el que el usuario visito el sitio
visitas = sets.loc[:,cols_visit]
features_usuario = visitas.loc[visitas.event == 'visited site']
features_usuario.drop(['event'], inplace=True, axis=1)
features_usuario['visitas_sitio'] = 1
features_usuario.head()

,fecha,person,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio
8,2018-05-18 00:07:22,4886f805,New,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1
480,2018-05-02 01:28:27,0297fc1e,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1
481,2018-05-02 14:28:20,0297fc1e,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1
482,2018-05-03 01:55:52,0297fc1e,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1
483,2018-05-10 00:38:17,0297fc1e,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1


In [28]:
features_usuario.count()

fecha                       101770
person                      101770
new_vs_returning            101770
region                      101770
country                     101770
device_type                 101770
screen_resolution           101769
operating_system_version    101770
browser_version             101770
visitas_sitio               101770
dtype: int64

In [37]:
print('1s ', otros[otros['label'] == 1].shape)
print('0s ', otros[otros['label'] == 0].shape)

1s  (46, 11)
0s  (522, 11)


In [38]:
# features_usuario = features_usuario.drop_duplicates()
# features_usuario.count()

In [29]:
del productos
del visitas

In [30]:
user_prod_comprados = productos_comprados.merge(features_usuario, on="person", how="left")
user_prod_comprados.head()

,fecha_x,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio
0,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,1,0,0,2018-05-18 00:07:22,New,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1.0
1,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,1,0,0,2018-05-02 01:28:27,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0
2,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,1,0,0,2018-05-02 14:28:20,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0
3,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,1,0,0,2018-05-03 01:55:52,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0
4,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,1,0,0,2018-05-10 00:38:17,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0


In [31]:
user_prod_comprados.count()

fecha_x                     564204
person                      564204
sku                         564204
model                       564204
condition                   564204
storage                     564204
color                       564204
compras_prod                564204
vistas_prod                 564204
otros                       564204
fecha_y                     563806
new_vs_returning            563806
region                      563806
country                     563806
device_type                 563806
screen_resolution           563805
operating_system_version    563806
browser_version             563806
visitas_sitio               563806
dtype: int64

In [42]:
print('1s_x ', user_prod_comprados[user_prod_comprados['label_x'] == 1].shape)
print('0s_x ', user_prod_comprados[user_prod_comprados['label_x'] == 0].shape)
print('1s_y ', user_prod_comprados[user_prod_comprados['label_y'] == 1].shape)
print('0s_y ', user_prod_comprados[user_prod_comprados['label_y'] == 0].shape)
print('1s_x y 1s_y', user_prod_comprados[(user_prod_comprados['label_x'] == 1) & (user_prod_comprados['label_y'] == 1)].shape)
print('0s_x y 0s_y', user_prod_comprados[(user_prod_comprados['label_x'] == 0) & (user_prod_comprados['label_y'] == 0)].shape)
print('1s_x y 0s_y', user_prod_comprados[(user_prod_comprados['label_x'] == 1) & (user_prod_comprados['label_y'] == 0)].shape)
print('0s_x y 1s_y ', user_prod_comprados[(user_prod_comprados['label_x'] == 0) & (user_prod_comprados['label_y'] == 1)].shape)

1s_x  (64127, 21)
0s_x  (78354, 21)
1s_y  (64119, 21)
0s_y  (77895, 21)
1s_x y 1s_y (64119, 21)
0s_x y 0s_y (77895, 21)
1s_x y 0s_y (0, 21)
0s_x y 1s_y  (0, 21)


In [32]:
user_prod_vistas = productos_visitados.merge(features_usuario, on="person", how="left")
user_prod_vistas.head()

,fecha_x,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio
0,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,0,1,0,2018-05-07 12:39:52,New,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,1.0
1,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,0,1,0,2018-05-07 12:43:42,Returning,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,1.0
2,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,0,1,0,2018-05-15 20:07:26,Returning,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,1.0
3,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,0,1,0,2018-05-18 00:21:05,Returning,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,1.0
4,2018-05-18 00:25:42,686c49c9,10086.0,iPhone 7,Excelente,128GB,Ouro Rosa,0,1,0,2018-05-18 01:05:32,Returning,Sao Paulo,Brazil,Smartphone,320x568,iOS 10.3.3,Mobile Safari 10,1.0


In [33]:
user_prod_vistas.count()

fecha_x                     627953
person                      627953
sku                         627953
model                       627953
condition                   627953
storage                     627953
color                       627953
compras_prod                627953
vistas_prod                 627953
otros                       627953
fecha_y                     627926
new_vs_returning            627926
region                      627926
country                     627926
device_type                 627926
screen_resolution           627926
operating_system_version    627926
browser_version             627926
visitas_sitio               627926
dtype: int64

In [45]:
print('1s_x ', user_prod_vistas[user_prod_vistas['label_x'] == 1].shape)
print('0s_x ', user_prod_vistas[user_prod_vistas['label_x'] == 0].shape)
print('1s_y ', user_prod_vistas[user_prod_vistas['label_y'] == 1].shape)
print('0s_y ', user_prod_vistas[user_prod_vistas['label_y'] == 0].shape)
print('1s_x y 1s_y', user_prod_vistas[(user_prod_vistas['label_x'] == 1) & (user_prod_vistas['label_y'] == 1)].shape)
print('0s_x y 0s_y', user_prod_vistas[(user_prod_vistas['label_x'] == 0) & (user_prod_vistas['label_y'] == 0)].shape)
print('1s_x y 0s_y', user_prod_vistas[(user_prod_vistas['label_x'] == 1) & (user_prod_vistas['label_y'] == 0)].shape)
print('0s_x y 1s_y ', user_prod_vistas[(user_prod_vistas['label_x'] == 0) & (user_prod_vistas['label_y'] == 1)].shape)

1s_x  (204354, 21)
0s_x  (97486, 21)
1s_y  (204244, 21)
0s_y  (97473, 21)
1s_x y 1s_y (204244, 21)
0s_x y 0s_y (97473, 21)
1s_x y 0s_y (0, 21)
0s_x y 1s_y  (0, 21)


In [34]:
otros = otros.merge(features_usuario, on="person", how="outer")
otros.head()

,fecha_x,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio
0,2018-05-28 12:36:14,df80a783,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,2018-05-28 12:36:14,New,Maranhao,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,1.0
1,2018-05-28 12:36:14,df80a783,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,2018-05-28 22:28:25,Returning,Minas Gerais,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,1.0
2,2018-05-28 12:36:14,df80a783,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,2018-05-29 14:47:28,Returning,Piaui,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,1.0
3,2018-05-28 12:36:14,df80a783,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,2018-05-29 17:00:15,Returning,Piaui,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,1.0
4,2018-05-28 22:28:24,df80a783,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,2018-05-28 12:36:14,New,Maranhao,Brazil,Smartphone,360x640,Android 5.1.1,Samsung Internet 6.4,1.0


In [35]:
otros.count()

fecha_x                       3877
person                      105273
sku                              0
model                            5
condition                        0
storage                          0
color                            0
compras_prod                  3877
vistas_prod                   3877
otros                         3877
fecha_y                     105260
new_vs_returning            105260
region                      105260
country                     105260
device_type                 105260
screen_resolution           105259
operating_system_version    105260
browser_version             105260
visitas_sitio               105260
dtype: int64

In [48]:
print('1s_x ', otros[otros['label_x'] == 1].shape)
print('0s_x ', otros[otros['label_x'] == 0].shape)
print('1s_y ', otros[otros['label_y'] == 1].shape)
print('0s_y ', otros[otros['label_y'] == 0].shape)
print('1s_x y 1s_y', otros[(otros['label_x'] == 1) & (otros['label_y'] == 1)].shape)
print('0s_x y 0s_y', otros[(otros['label_x'] == 0) & (otros['label_y'] == 0)].shape)
print('1s_x y 0s_y', otros[(otros['label_x'] == 1) & (otros['label_y'] == 0)].shape)
print('0s_x y 1s_y ', otros[(otros['label_x'] == 0) & (otros['label_y'] == 1)].shape)

1s_x  (63, 21)
0s_x  (3246, 21)
1s_y  (8956, 21)
0s_y  (45423, 21)
1s_x y 1s_y (63, 21)
0s_x y 0s_y (3238, 21)
1s_x y 0s_y (0, 21)
0s_x y 1s_y  (0, 21)


In [36]:
del features_usuario
del productos_comprados
del productos_visitados

In [37]:
#concateno
frames = [user_prod_comprados, user_prod_vistas, otros]
sets = pd.concat(frames)
sets.head()

,fecha_x,person,sku,model,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio
0,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,1.0,0.0,0.0,2018-05-18 00:07:22,New,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1.0
1,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-02 01:28:27,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0
2,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-02 14:28:20,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0
3,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-03 01:55:52,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0
4,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-10 00:38:17,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0


In [38]:
del frames
del user_prod_comprados
del user_prod_vistas
del otros

In [39]:
sets.count()

fecha_x                     1196034
person                      1297430
sku                         1192157
model                       1192162
condition                   1192157
storage                     1192157
color                       1192157
compras_prod                1196034
vistas_prod                 1196034
otros                       1196034
fecha_y                     1296992
new_vs_returning            1296992
region                      1296992
country                     1296992
device_type                 1296992
screen_resolution           1296990
operating_system_version    1296992
browser_version             1296992
visitas_sitio               1296992
dtype: int64

In [40]:
# sets = sets.drop_duplicates()

In [41]:
# Rellenamos compras_prod  vistas_prod  otros y visitas_sitio con 0
sets['compras_prod'] = sets['compras_prod'].fillna(value=0) 
sets['vistas_prod'] = sets['vistas_prod'].fillna(value=0) 
sets['otros'] = sets['otros'].fillna(value=0) 
sets['visitas_sitio'] = sets['visitas_sitio'].fillna(value=0) 

In [42]:
sets.count()

fecha_x                     1196034
person                      1297430
sku                         1192157
model                       1192162
condition                   1192157
storage                     1192157
color                       1192157
compras_prod                1297430
vistas_prod                 1297430
otros                       1297430
fecha_y                     1296992
new_vs_returning            1296992
region                      1296992
country                     1296992
device_type                 1296992
screen_resolution           1296990
operating_system_version    1296992
browser_version             1296992
visitas_sitio               1297430
dtype: int64

In [43]:
# Rellenamos label_x y label_y con 0 y nos quedamos con label
sets['label_x'] = sets['label_x'].fillna(value=0) 
sets['label_y'] = sets['label_y'].fillna(value=0) 
sets.loc[(sets['label_x'] == 1) | (sets['label_y'] == 1), 'label'] = 1
sets['label'] = sets['label'].fillna(value=0) 
sets.head()

KeyError: 'label_x'

In [57]:
print('1s ', sets[sets['label'] == 1].shape)
print('0s ',sets[sets['label'] == 0].shape)

1s  (277437, 22)
0s  (221271, 22)


In [58]:
sets = sets.drop('label_x',1)
sets = sets.drop('label_y',1)
sets.count()

browser_version             498110
color                       444321
compras_prod                498708
condition                   444321
country                     498110
device_type                 498110
fecha_x                     447630
fecha_y                     498110
model                       444323
new_vs_returning            498110
operating_system_version    498110
otros                       498708
person                      498708
region                      498110
screen_resolution           498097
sku                         444321
storage                     444321
visitas_sitio               498708
vistas_prod                 498708
label                       498708
dtype: int64

In [44]:
# Rellenamos model con 0 para luego hacer un split entre marca y modelo
sets['model'] = sets['model'].fillna(value='0') 

In [45]:
# Rellenamos los demas con ''
sets = sets.fillna(value='') 

In [46]:
sets.count()

fecha_x                     1297430
person                      1297430
sku                         1297430
model                       1297430
condition                   1297430
storage                     1297430
color                       1297430
compras_prod                1297430
vistas_prod                 1297430
otros                       1297430
fecha_y                     1297430
new_vs_returning            1297430
region                      1297430
country                     1297430
device_type                 1297430
screen_resolution           1297430
operating_system_version    1297430
browser_version             1297430
visitas_sitio               1297430
dtype: int64

In [47]:
sets.person.describe()

count      1297430
unique       19415
top       5059f7fd
freq         96465
Name: person, dtype: object

## Categorizacion de los features

In [48]:
# Partimos el feature model en marca y modelo
sets['model'] = sets['model'].apply(lambda x: x.split(' '))
sets['marca'] = sets['model'].apply(lambda x: "apple" if str(x[0]) == "iPhone" else str(x[0]))
sets['modelo'] = sets['model'].apply(lambda x: ' '.join(x[1:]) if len(x)>1 else "0" )
sets.drop('model', axis=1, inplace=True)
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,new_vs_returning,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio,marca,modelo
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1.0,0.0,0.0,2018-05-18 00:07:22,New,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1.0,Samsung,Galaxy J7 Prime
1,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-02 01:28:27,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S
2,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-02 14:28:20,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S
3,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-03 01:55:52,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S
4,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-10 00:38:17,Returning,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S


In [49]:
# creo una nueva columna person_int para entrenar transformando el valor alfanumerico de person a numerico
sets['person_int'] = labelencoder.fit_transform(sets['person'])
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1.0,0.0,0.0,2018-05-18 00:07:22,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1.0,Samsung,Galaxy J7 Prime,5476
1,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-02 01:28:27,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
2,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-02 14:28:20,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
3,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-03 01:55:52,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
4,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1.0,0.0,0.0,2018-05-10 00:38:17,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189


In [50]:
# convertimos a integer el valor flotante de compras_prod
sets['compras_prod'] = sets['compras_prod'].astype('int')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1,0.0,0.0,2018-05-18 00:07:22,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1.0,Samsung,Galaxy J7 Prime,5476
1,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0.0,0.0,2018-05-02 01:28:27,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
2,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0.0,0.0,2018-05-02 14:28:20,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
3,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0.0,0.0,2018-05-03 01:55:52,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
4,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0.0,0.0,2018-05-10 00:38:17,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189


In [51]:
# convertimos a integer el valor flotante de vistas_prod
sets['vistas_prod'] = sets['vistas_prod'].astype('int')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1,0,0.0,2018-05-18 00:07:22,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1.0,Samsung,Galaxy J7 Prime,5476
1,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0.0,2018-05-02 01:28:27,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
2,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0.0,2018-05-02 14:28:20,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
3,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0.0,2018-05-03 01:55:52,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
4,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0.0,2018-05-10 00:38:17,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189


In [52]:
# convertimos a integer el valor flotante de otros
sets['otros'] = sets['otros'].astype('int')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1,0,0,2018-05-18 00:07:22,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1.0,Samsung,Galaxy J7 Prime,5476
1,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0,2018-05-02 01:28:27,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
2,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0,2018-05-02 14:28:20,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
3,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0,2018-05-03 01:55:52,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189
4,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0,2018-05-10 00:38:17,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1.0,apple,6S,189


In [53]:
# convertimos a integer el valor flotante de visitas_sitio
sets['visitas_sitio'] = sets['visitas_sitio'].astype('int')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288,Excelente,32GB,Dourado,1,0,0,2018-05-18 00:07:22,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1,Samsung,Galaxy J7 Prime,5476
1,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0,2018-05-02 01:28:27,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1,apple,6S,189
2,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0,2018-05-02 14:28:20,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1,apple,6S,189
3,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0,2018-05-03 01:55:52,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1,apple,6S,189
4,2018-05-22 20:29:35,0297fc1e,8992,Bom,32GB,Ouro Rosa,1,0,0,2018-05-10 00:38:17,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1,apple,6S,189


In [54]:
# convertimos a integer el valor flotante de label
sets['label'] = sets['label'].astype('int')
sets.head()

KeyError: 'label'

In [55]:
# convertimos a string el valor flotante de sku
sets['sku'] = sets['sku'].astype('str')
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,9288.0,Excelente,32GB,Dourado,1,0,0,2018-05-18 00:07:22,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,1,Samsung,Galaxy J7 Prime,5476
1,2018-05-22 20:29:35,0297fc1e,8992.0,Bom,32GB,Ouro Rosa,1,0,0,2018-05-02 01:28:27,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1,apple,6S,189
2,2018-05-22 20:29:35,0297fc1e,8992.0,Bom,32GB,Ouro Rosa,1,0,0,2018-05-02 14:28:20,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1,apple,6S,189
3,2018-05-22 20:29:35,0297fc1e,8992.0,Bom,32GB,Ouro Rosa,1,0,0,2018-05-03 01:55:52,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1,apple,6S,189
4,2018-05-22 20:29:35,0297fc1e,8992.0,Bom,32GB,Ouro Rosa,1,0,0,2018-05-10 00:38:17,...,Rio de Janeiro,Brazil,Smartphone,360x640,Android 6.0.1,Chrome Mobile 65.0,1,apple,6S,189


In [56]:
#Categorizamos los valores de los features con label encoder
sets['sku'] = labelencoder.fit_transform(sets['sku'])
sets['marca'] = labelencoder.fit_transform(sets['marca'])
sets['modelo'] = labelencoder.fit_transform(sets['modelo'])
sets['condition'] = labelencoder.fit_transform(sets['condition'])
sets['storage'] = labelencoder.fit_transform(sets['storage'])
sets['color'] = labelencoder.fit_transform(sets['color'])
sets['new_vs_returning'] = labelencoder.fit_transform(sets['new_vs_returning'])
sets['region'] = labelencoder.fit_transform(sets['region'])
sets['country'] = labelencoder.fit_transform(sets['country'])
sets['device_type'] = labelencoder.fit_transform(sets['device_type'])
sets['screen_resolution'] = labelencoder.fit_transform(sets['screen_resolution'])
sets['operating_system_version'] = labelencoder.fit_transform(sets['operating_system_version'])
sets['browser_version'] = labelencoder.fit_transform(sets['browser_version'])
sets.head()

,fecha_x,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,fecha_y,...,region,country,device_type,screen_resolution,operating_system_version,browser_version,visitas_sitio,marca,modelo,person_int
0,2018-05-18 00:11:56,4886f805,1622,3,4,28,1,0,0,2018-05-18 00:07:22,...,78,7,2,170,24,110,1,6,67,5476
1,2018-05-22 20:29:35,0297fc1e,1572,1,4,34,1,0,0,2018-05-02 01:28:27,...,78,7,2,170,22,111,1,8,13,189
2,2018-05-22 20:29:35,0297fc1e,1572,1,4,34,1,0,0,2018-05-02 14:28:20,...,78,7,2,170,22,111,1,8,13,189
3,2018-05-22 20:29:35,0297fc1e,1572,1,4,34,1,0,0,2018-05-03 01:55:52,...,78,7,2,170,22,111,1,8,13,189
4,2018-05-22 20:29:35,0297fc1e,1572,1,4,34,1,0,0,2018-05-10 00:38:17,...,78,7,2,170,22,111,1,8,13,189


In [57]:
#Categorizamos la fecha del visto del producto en dia mes y dia de la semana  (ya que el anio es 2018)
sets['dia_view_prod'] = sets['fecha_x'].apply(lambda x: x.day if type(x) != str else 0).astype('int')
sets['mes_view_prod'] = sets['fecha_x'].apply(lambda x: x.month if type(x) != str else 0).astype('int')
sets['dia_sem_view_prod'] = sets['fecha_x'].apply(lambda x: x.dayofweek if type(x) != str else 0).astype('int')
sets = sets.drop('fecha_x',1)

In [58]:
#Categorizamos la fecha de visita del usuario en dia mes y dia de la semana (ya que el anio es 2018)
sets['dia_visit_user'] = sets['fecha_y'].apply(lambda x: x.day if type(x) != str else 0).astype('int')
sets['mes_visit_user'] = sets['fecha_y'].apply(lambda x: x.month if type(x) != str else 0).astype('int')
sets['dia_sem_visit_user'] = sets['fecha_y'].apply(lambda x: x.dayofweek if type(x) != str else 0).astype('int')
sets = sets.drop('fecha_y',1)

In [59]:
sets.head()

,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,new_vs_returning,region,...,visitas_sitio,marca,modelo,person_int,dia_view_prod,mes_view_prod,dia_sem_view_prod,dia_visit_user,mes_visit_user,dia_sem_visit_user
0,4886f805,1622,3,4,28,1,0,0,1,78,...,1,6,67,5476,18,5,4,18,5,4
1,0297fc1e,1572,1,4,34,1,0,0,2,78,...,1,8,13,189,22,5,1,2,5,2
2,0297fc1e,1572,1,4,34,1,0,0,2,78,...,1,8,13,189,22,5,1,2,5,2
3,0297fc1e,1572,1,4,34,1,0,0,2,78,...,1,8,13,189,22,5,1,3,5,3
4,0297fc1e,1572,1,4,34,1,0,0,2,78,...,1,8,13,189,22,5,1,10,5,3


In [60]:
sets.tail()

,person,sku,condition,storage,color,compras_prod,vistas_prod,otros,new_vs_returning,region,...,visitas_sitio,marca,modelo,person_int,dia_view_prod,mes_view_prod,dia_sem_view_prod,dia_visit_user,mes_visit_user,dia_sem_visit_user
105268,fb88a7ea,0,0,0,0,0,0,0,1,15,...,1,0,1,19100,0,0,0,16,2,4
105269,9707cd0e,0,0,0,0,0,0,0,1,82,...,1,0,1,11464,0,0,0,30,5,2
105270,6f7632db,0,0,0,0,0,0,0,1,90,...,1,0,1,8449,0,0,0,21,5,0
105271,a1c2a901,0,0,0,0,0,0,0,1,84,...,1,0,1,12257,0,0,0,17,5,3
105272,ed3f80d7,0,0,0,0,0,0,0,1,82,...,1,0,1,18012,0,0,0,18,4,2


In [61]:
sets.count()

person                      1297430
sku                         1297430
condition                   1297430
storage                     1297430
color                       1297430
compras_prod                1297430
vistas_prod                 1297430
otros                       1297430
new_vs_returning            1297430
region                      1297430
country                     1297430
device_type                 1297430
screen_resolution           1297430
operating_system_version    1297430
browser_version             1297430
visitas_sitio               1297430
marca                       1297430
modelo                      1297430
person_int                  1297430
dia_view_prod               1297430
mes_view_prod               1297430
dia_sem_view_prod           1297430
dia_visit_user              1297430
mes_visit_user              1297430
dia_sem_visit_user          1297430
dtype: int64

In [62]:
sets.person.describe()

count      1297430
unique       19415
top       5059f7fd
freq         96465
Name: person, dtype: object

In [78]:
sets[sets['label'] == 1].shape

(277437, 26)

In [79]:
sets[sets['label'] == 0].shape

(221271, 26)

In [66]:
# Guardar Set

In [63]:
sets.to_csv('data/set_prediccion_17.csv', encoding='utf-8', index=False)

In [80]:
sets.to_csv('data/set_entrenamiento_17.csv', encoding='utf-8', index=False)